In [44]:
using DelimitedFiles, Dierckx, Plots, GridInterpolations

In [16]:
FILE_F_MAP = "../data/F_REFERENCE.txt"
FILE_PS = "../data/WideA_ZA_pk.dat"
FILE_BACKGROUND = "../data/WideA_ZA_background.dat";

In [34]:
struct WindowF
     xs::AbstractVector{T1} where {T1}
     μs::AbstractVector{T2} where {T2}
     Fs::AbstractArray{T3,2} where {T3}

     function WindowF(file::String)
          data = readdlm(file, comments = true)
          xs, μs, Fs = data[:, 1], data[:, 2], data[:, 3]
          @assert size(xs) == size(μs) == size(Fs) "xs, μs and Fs must have the same length!"

          new_xs = unique(xs)
          new_μs = unique(μs)
          new_Fs =
               if xs[2] == xs[1] && μs[2] ≠ μs[1]
                    transpose(reshape(Fs, (length(new_μs), length(new_xs))))
               elseif xs[2] ≠ xs[1] && μs[2] == μs[1]
                    reshape(Fs, (length(new_xs), length(new_μs)))
               else
                    throw(ErrorException("What kind of convenction for the file $file" *
                                         " are you using? I do not recognise it."))
               end
        grid = GridInterpolations.RectangleGrid(new_μs, new_xs)
        spline(x, μ) = GridInterpolations.interpolate(grid, new_Fs, [x, μ])
          new(new_xs, new_μs, new_Fs, spline)
     end
end

In [35]:
str = new_WindowF("F_REFERENCE.txt")

new_WindowF([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9  …  2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0], [-1.0, -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92, -0.91  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], [0.0 20.315581703061408 … 20.315581372967298 0.0; 0.0 20.121689187178195 … 19.984420159530657 0.0; … ; 0.0 1.7647802990836603 … 18.563437720176587 0.0; 0.0 1.7647802990836603 … 18.563437720176587 0.0], var"#spline#18"{RectangleGrid{2}, LinearAlgebra.Transpose{Float64, Matrix{Float64}}}(RectangleGrid{2}([-1.0, -0.99, -0.98, -0.97, -0.96, -0.95, -0.94, -0.93, -0.92, -0.91  …  0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0],[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9  …  2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0],), [0.0 20.315581703061408 … 20.315581372967298 0.0; 0.0 20.121689187178195 … 19.984420159530657 0.0; … ; 0.0 1.7647802990836603 … 18.563437720176587 0.0; 0.0 1.7647802990836603 … 18.563437720176587 0.0]))

In [42]:
reshape(str.Fs, (:, 1))

1209×1 reshape(transpose(::Matrix{Float64}), 1209, 1) with eltype Float64:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [38]:
grid = RectangleGrid(str.xs , str.μs)
f(x, μ) = GridInterpolations.interpolate(grid, str.Fs, [x, μ])

f (generic function with 1 method)

In [39]:
plot(str.xs, str.μs, f; 
    xl="x", yl="y", zl="z", st = :surface, 
    c=cgrad([:yellow, :red]), size=(300, 300))

LoadError: MethodError: no method matching interpolate(::RectangleGrid{2}, ::LinearAlgebra.Transpose{Float64, Matrix{Float64}}, ::Vector{Float64})
[0mClosest candidates are:
[0m  interpolate(::AbstractGrid, [91m::Matrix[39m, ::AbstractVector) at ~/.julia/packages/GridInterpolations/Q9kj8/src/GridInterpolations.jl:158
[0m  interpolate(::AbstractGrid, [91m::DenseArray[39m, ::AbstractVector) at ~/.julia/packages/GridInterpolations/Q9kj8/src/GridInterpolations.jl:160

In [43]:
struct tipo{Union{Dierckx.Spline1D}}
    a::T
end



LoadError: syntax: invalid variable expression in "where" around In[43]:1